<a href="https://colab.research.google.com/github/kjsampler/Using-AI-to-Predict-Dog-Licenses/blob/main/Multivariate_Prediction_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rather than clutter another notebook, I've decided to show the process of writing the multivariate prediction function in its own seperate notebook. This will allow for more organization and a better ability to explain the process. 